In [5]:
import ctypes
import os
import time
import sys

def check(err):
    if err != 0:
        raise Exception("Error: 0x{:08X}".format(err))

# -----------------------------------------------------------------------------
# 1. Load the EDSDK Library (update the path as necessary)
# -----------------------------------------------------------------------------
if sys.platform == 'darwin':
    edsdk = ctypes.CDLL('CANON_SDK/EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('EDSDK.dll')

# -----------------------------------------------------------------------------
# 2. Define Basic Types and Constants
# -----------------------------------------------------------------------------
CameraRef = ctypes.c_void_p
DirectoryItemRef = ctypes.c_void_p
StreamRef = ctypes.c_void_p

EDS_MAX_NAME = 256

# Shutter Command Constants (verify these with your EDSDK headers)
kEdsCameraCommand_PressShutterButton = 0x00000000
kEdsCameraCommand_ShutterButton_Completely = 0x00000001
kEdsCameraCommand_ShutterButton_OFF = 0x00000000


# File stream constants (verify in the SDK documentation)
kEdsFileCreateDisposition_CreateAlways = 2
kEdsAccess_ReadWrite = 2

# -----------------------------------------------------------------------------
# 3. Define Function Prototypes
# -----------------------------------------------------------------------------
edsdk.EdsInitializeSDK.argtypes = []
edsdk.EdsInitializeSDK.restype = ctypes.c_int

edsdk.EdsTerminateSDK.argtypes = []
edsdk.EdsTerminateSDK.restype = ctypes.c_int

edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetCameraList.restype = ctypes.c_int

edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint)]
edsdk.EdsGetChildCount.restype = ctypes.c_int

edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_uint, ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetChildAtIndex.restype = ctypes.c_int

edsdk.EdsOpenSession.argtypes = [CameraRef]
edsdk.EdsOpenSession.restype = ctypes.c_int

edsdk.EdsCloseSession.argtypes = [CameraRef]
edsdk.EdsCloseSession.restype = ctypes.c_int

edsdk.EdsSendCommand.argtypes = [CameraRef, ctypes.c_uint, ctypes.c_uint]
edsdk.EdsSendCommand.restype = ctypes.c_int

edsdk.EdsCreateFileStream.argtypes = [ctypes.c_char_p, ctypes.c_uint, ctypes.c_uint, ctypes.POINTER(StreamRef)]
edsdk.EdsCreateFileStream.restype = ctypes.c_int

edsdk.EdsDownload.argtypes = [DirectoryItemRef, ctypes.c_uint, StreamRef]
edsdk.EdsDownload.restype = ctypes.c_int

edsdk.EdsDownloadComplete.argtypes = [DirectoryItemRef]
edsdk.EdsDownloadComplete.restype = ctypes.c_int

class EdsDirectoryItemInfo(ctypes.Structure):
    _fields_ = [
        ("size", ctypes.c_uint32),                      # File size in bytes.
        ("isFolder", ctypes.c_uint32),                  # Non-zero if the object is a folder.
        ("szFileName", ctypes.c_char * EDS_MAX_NAME)      # File name (C string).
        # There are additional fields in the full structure if needed.
    ]



In [6]:

# -----------------------------------------------------------------------------
# 4. Main Capture Process (Using Folder at Index 0)
# -----------------------------------------------------------------------------
try:
    # Initialize the SDK.
    check(edsdk.EdsInitializeSDK())
    print("EDSDK initialized.")

    # Retrieve the camera list.
    camList = ctypes.c_void_p()
    check(edsdk.EdsGetCameraList(ctypes.byref(camList)))
    
    # Get camera count.
    camCount = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(camList, ctypes.byref(camCount)))
    if camCount.value == 0:
        raise Exception("No cameras found!")
    print("Found {} camera(s).".format(camCount.value))

    # Get the first camera.
    camera = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(camList, 0, ctypes.byref(camera)))
    print("Camera obtained.")

except Exception as e:
    print("An error occured: ", e)

# finally:
#     try:
#         check(edsdk.EdsTerminateSDK())
#         print("EDSDK terminated.")
#     except Exception:
#         pass

# except Exception as e:
#     print("An error occurred:", e)


EDSDK initialized.
Found 1 camera(s).
Camera obtained.


In [9]:
try:
    # Open a session with the camera.
    check(edsdk.EdsOpenSession(camera))
    print("Session opened with camera.")

    # Trigger a capture on the camera.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_Completely))
    print("Capture triggered.")

    # Close the session.
    check(edsdk.EdsCloseSession(camera))
    print("Session closed.")

finally:
    try:
        # check(edsdk.EdsTerminateSDK())
        check(edsdk.EdsCloseSession(camera))
        print("EDSDK terminated.")
    except Exception:
        pass

Session opened with camera.
Capture triggered.
Session closed.


In [10]:
try:    
    # Open a session with the camera.
    check(edsdk.EdsOpenSession(camera))
    print("Session opened with camera.")

    # Obtain the SD card volume.
    volume = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(camera, 0, ctypes.byref(volume)))
    print("Volume (SD card) obtained.")

    # Directly get the folder at index 0.
    dcim_folder = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(volume, 0, ctypes.byref(dcim_folder)))
    print("DCIM obtained.")

    canon100_folder = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(dcim_folder, 0, ctypes.byref(canon100_folder)))
    print("canon100 folder obtained.")


    # Record baseline file count in folder.
    image_count = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(canon100_folder, ctypes.byref(image_count)))
    print("Initial file count in folder:", image_count.value)

    
    file_index = image_count.value - 1
    file_ref = CameraRef()
    err = edsdk.EdsGetChildAtIndex(canon100_folder, file_index, ctypes.byref(file_ref))
    check(err)
    print("Most recent file reference obtained.")


    dir_info = EdsDirectoryItemInfo()
    err = edsdk.EdsGetDirectoryItemInfo(file_ref, ctypes.byref(dir_info))
    check(err)
    file_size = dir_info.size
    file_name = dir_info.szFileName.decode('utf-8', errors='ignore')
    print(f"File Info: {file_name} ({file_size} bytes).")



    local_filename = b"downloaded_image.jpg"  # Byte string for ctypes.

    with open(local_filename, "w") as file:
        file.write("")

    file_stream = StreamRef()
    err = edsdk.EdsCreateFileStream(local_filename,
                                    kEdsFileCreateDisposition_CreateAlways,
                                    kEdsAccess_ReadWrite,
                                    ctypes.byref(file_stream))
    check(err)
    print(f"Local file stream created: {local_filename.decode()}")


    err = edsdk.EdsDownload(file_ref, file_size, file_stream)
    check(err)
    err = edsdk.EdsDownloadComplete(file_ref)
    check(err)
    print("Download complete.")

    check(edsdk.EdsCloseSession(camera))

except Exception as e:
    print("Exception: ", e)

Session opened with camera.
Volume (SD card) obtained.
DCIM obtained.
canon100 folder obtained.
Initial file count in folder: 104
Most recent file reference obtained.
File Info:  (23246365 bytes).
Local file stream created: downloaded_image.jpg
Download complete.


In [5]:
try:
    check(edsdk.EdsTerminateSDK())
    # check(edsdk.EdsCloseSession(camera))
    print("EDSDK terminated.")
except Exception:
    pass

EDSDK terminated.
